# Local RAG agent with LLaMA3

In [47]:
# Some pip installs
# %%capture --no-stderr

%pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4

Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


In [48]:
# Embeddings: GPT4ALLEmbeddings
%pip install langchain-nomic

# LLM using ollama and llama3.2
%ollama pull llama3.2:3b-instructfp16

Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>UsageError: Line magic function `%ollama` not found.



In [22]:
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Model Descriptor                        | Hugging Face Repo                                   | Context Length |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-8B                             | meta-llama/Llama-3.1-8B                             | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-70B                            | meta-llama/Llama-3.1-70B                            | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-405B:bf16-mp8                  | meta-llama/Llama-3.1-405B                           | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-405B                           | meta-llama/Llama-3.1-405B-FP8                       | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-405B:bf16-mp16                 | meta-llama/Llama-3.1-405B                           | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-8B-Instruct                    | meta-llama/Llama-3.1-8B-Instruct                    | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-70B-Instruct                   | meta-llama/Llama-3.1-70B-Instruct                   | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-405B-Instruct:bf16-mp8         | meta-llama/Llama-3.1-405B-Instruct                  | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-405B-Instruct                  | meta-llama/Llama-3.1-405B-Instruct-FP8              | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.1-405B-Instruct:bf16-mp16        | meta-llama/Llama-3.1-405B-Instruct                  | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-1B                             | meta-llama/Llama-3.2-1B                             | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-3B                             | meta-llama/Llama-3.2-3B                             | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-11B-Vision                     | meta-llama/Llama-3.2-11B-Vision                     | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-90B-Vision                     | meta-llama/Llama-3.2-90B-Vision                     | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-1B-Instruct                    | meta-llama/Llama-3.2-1B-Instruct                    | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-3B-Instruct                    | meta-llama/Llama-3.2-3B-Instruct                    | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-1B-Instruct:int4-qlora-eo8     | meta-llama/Llama-3.2-1B-Instruct-QLORA_INT4_EO8     | 8K             |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-1B-Instruct:int4-spinquant-eo8 | meta-llama/Llama-3.2-1B-Instruct-SpinQuant_INT4_EO8 | 8K             |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-3B-Instruct:int4-qlora-eo8     | meta-llama/Llama-3.2-3B-Instruct-QLORA_INT4_EO8     | 8K             |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-3B-Instruct:int4-spinquant-eo8 | meta-llama/Llama-3.2-3B-Instruct-SpinQuant_INT4_EO8 | 8K             |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-11B-Vision-Instruct            | meta-llama/Llama-3.2-11B-Vision-Instruct            | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama3.2-90B-Vision-Instruct            | meta-llama/Llama-3.2-90B-Vision-Instruct            | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama-Guard-3-11B-Vision                | meta-llama/Llama-Guard-3-11B-Vision                 | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama-Guard-3-1B:int4                   | meta-llama/Llama-Guard-3-1B-INT4                    | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama-Guard-3-1B                        | meta-llama/Llama-Guard-3-1B                         | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama-Guard-3-8B                        | meta-llama/Llama-Guard-3-8B                         | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama-Guard-3-8B:int8                   | meta-llama/Llama-Guard-3-8B-INT8                    | 128K           |
# +-----------------------------------------+-----------------------------------------------------+----------------+
# | Llama-Guard-2-8B                        | meta-llama/Llama-Guard-2-8B                         | 4K             |
# +-----------------------------------------+-----------------------------------------------------+----------------+

In [ ]:
# We need to run the llama cli to download the model
%pip install llama-stack
# Note: llama commands will not run in the jupyter notebook, they need to run on the terminal
# Lets list the models available
# %llama model list
# download the desired model
desired_model = "Llama3.2-1B-Instruct"
# %llama download --source meta --model-id desired_model
# % llama download --source meta --mod

%pip install torch fairscale fire blobfile

# LLM

In [49]:
### LLM (ollama)
from langchain_ollama import ChatOllama

local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [70]:
# From huggin face
# %huggingface-cli download meta-llama/Meta-Llama-3.1-8B-Instruct --include "original/*" --local-dir meta-llama/Meta-Llama-3.1-8B-Instruct
# $ huggingface-cli login
# LLM using ollama and llama3.2
# %ollama pull llama3.2:3b-instructfp16
# %pip install accelerate>=0.26.0
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    # device_map="auto",  # may need accelerate or could crash
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
# For local llm, not working because of modle download issues.
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
# Replace with your actual model path
# model_path = "models/llama3_2"  # or "/full/path/to/llama3.2" if it's outside your project
model_path = "models/Llama3.2-1B-Instruct"

if os.path.exists(model_path) and os.path.isdir(model_path):
    print(f"The path '{model_path}' is valid")
else:
    print(f"The path '{model_path}' is not valid.")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test it with a sample input
def generate_response(prompt, temperature=0.0, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, temperature=temperature)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
prompt = "What is the weather like today?"
response = generate_response(prompt)
print(response)
# Local LLM
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "./models/llama3_2"

# Load the tokenizer and model

# Search

In [42]:
# Search with tavily
import os
import getpass
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

# Retrieve the TAVILY_API_KEY from environment variables, or prompt if not found
tavily_api_key = os.getenv("TAVILY_API_KEY")
if not tavily_api_key:
    tavily_api_key = getpass.getpass("TAVILY_API_KEY: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

# Additional environment settings
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Tracing

In [51]:
# I dont actually want to use tracing
# _set_env("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "local-llama32-rag"

# Vectorstore

In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

# Create retriever
retriever = vectorstore.as_retriever(k=3)

# Components
## Router

In [54]:
### Router
import json
from langchain_core.messages import HumanMessage, SystemMessage

# Prompt
router_instructions = """You are an expert at routing a user question to a vectorstore or web search.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.

Return JSON with single key, datasource, that is 'websearch' or 'vectorstore' depending on the question."""

# Test router
test_web_search = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [
        HumanMessage(
            content="Who is favored to win the NFC Championship game in the 2024 season?"
        )
    ]
)
test_web_search_2 = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [HumanMessage(content="What are the models released today for llama3.2?")]
)
test_vector_store = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [HumanMessage(content="What are the types of agent memory?")]
)
print(
    json.loads(test_web_search.content),
    json.loads(test_web_search_2.content),
    json.loads(test_vector_store.content),
)

ResponseError: model "llama3.2:3b-instruct-fp16" not found, try pulling it first

## Retrieval Grader

In [ ]:
### Retrieval Grader

# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

# Test
question = "What is Chain of thought prompting?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)
json.loads(result.content)

In [27]:
## Generater

In [ ]:
### Generate

# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

In [28]:
## Hallucination Grader

In [ ]:
### Hallucination Grader

# Hallucination grader instructions
hallucination_grader_instructions = """

You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 

(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader prompt
hallucination_grader_prompt = """FACTS: \n\n {documents} \n\n STUDENT ANSWER: {generation}. 

Return JSON with two two keys, binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER is grounded in the FACTS. And a key, explanation, that contains an explanation of the score."""

# Test using documents and generation from above
hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
    documents=docs_txt, generation=generation.content
)
result = llm_json_mode.invoke(
    [SystemMessage(content=hallucination_grader_instructions)]
    + [HumanMessage(content=hallucination_grader_prompt_formatted)]
)
json.loads(result.content)

In [ ]:
### Answer Grader

# Answer grader instructions
answer_grader_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) The STUDENT ANSWER helps to answer the QUESTION

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

The student can receive a score of yes if the answer contains extra information that is not explicitly asked for in the question.

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader prompt
answer_grader_prompt = """QUESTION: \n\n {question} \n\n STUDENT ANSWER: {generation}. 

Return JSON with two two keys, binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER meets the criteria. And a key, explanation, that contains an explanation of the score."""

# Test
question = "What are the vision models released today as part of Llama 3.2?"
answer = "The Llama 3.2 models released today include two vision models: Llama 3.2 11B Vision Instruct and Llama 3.2 90B Vision Instruct, which are available on Azure AI Model Catalog via managed compute. These models are part of Meta's first foray into multimodal AI and rival closed models like Anthropic's Claude 3 Haiku and OpenAI's GPT-4o mini in visual reasoning. They replace the older text-only Llama 3.1 models."

# Test using question and generation from above
answer_grader_prompt_formatted = answer_grader_prompt.format(
    question=question, generation=answer
)
result = llm_json_mode.invoke(
    [SystemMessage(content=answer_grader_instructions)]
    + [HumanMessage(content=answer_grader_prompt_formatted)]
)
json.loads(result.content)

In [ ]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [ ]:
import operator
from typing_extensions import TypedDict
from typing import List, Annotated


class GraphState(TypedDict):
    """
    Graph state is a dictionary that contains information we want to propagate to, and modify in, each graph node.
    """

    question: str  # User question
    generation: str  # LLM generation
    web_search: str  # Binary decision to run web search
    max_retries: int  # Max number of retries for answer generation
    answers: int  # Number of answers generated
    loop_step: Annotated[int, operator.add]
    documents: List[str]  # List of retrieved documents

In [ ]:
from langchain.schema import Document
from langgraph.graph import END


### Nodes
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Write retrieved documents to documents key in state
    documents = retriever.invoke(question)
    return {"documents": documents}


def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    loop_step = state.get("loop_step", 0)

    # RAG generation
    docs_txt = format_docs(documents)
    rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
    generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
    return {"generation": generation, "loop_step": loop_step + 1}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        doc_grader_prompt_formatted = doc_grader_prompt.format(
            document=d.page_content, question=question
        )
        result = llm_json_mode.invoke(
            [SystemMessage(content=doc_grader_instructions)]
            + [HumanMessage(content=doc_grader_prompt_formatted)]
        )
        grade = json.loads(result.content)["binary_score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "web_search": web_search}


def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)
    return {"documents": documents}


### Edges


def route_question(state):
    """
    Route question to web search or RAG

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    route_question = llm_json_mode.invoke(
        [SystemMessage(content=router_instructions)]
        + [HumanMessage(content=state["question"])]
    )
    source = json.loads(route_question.content)["datasource"]
    if source == "websearch":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: NOT ALL DOCUMENTS ARE RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    max_retries = state.get("max_retries", 3)  # Default to 3 if not provided

    hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
        documents=format_docs(documents), generation=generation.content
    )
    result = llm_json_mode.invoke(
        [SystemMessage(content=hallucination_grader_instructions)]
        + [HumanMessage(content=hallucination_grader_prompt_formatted)]
    )
    grade = json.loads(result.content)["binary_score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        # Test using question and generation from above
        answer_grader_prompt_formatted = answer_grader_prompt.format(
            question=question, generation=generation.content
        )
        result = llm_json_mode.invoke(
            [SystemMessage(content=answer_grader_instructions)]
            + [HumanMessage(content=answer_grader_prompt_formatted)]
        )
        grade = json.loads(result.content)["binary_score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        elif state["loop_step"] <= max_retries:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
        else:
            print("---DECISION: MAX RETRIES REACHED---")
            return "max retries"
    elif state["loop_step"] <= max_retries:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"
    else:
        print("---DECISION: MAX RETRIES REACHED---")
        return "max retries"

In [ ]:
from langgraph.graph import StateGraph
from IPython.display import Image, display

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
        "max retries": END,
    },
)

# Compile
graph = workflow.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
inputs = {"question": "What are the types of agent memory?", "max_retries": 3}
for event in graph.stream(inputs, stream_mode="values"):
    print(event)

In [ ]:
# Test on current events
inputs = {
    "question": "What are the models released today for llama3.2?",
    "max_retries": 3,
}
for event in graph.stream(inputs, stream_mode="values"):
    print(event)